In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
from statsmodels.regression.rolling import RollingOLS

In [2]:
his_members = pd.read_excel("../data/members.xlsx", sheet_name=1)
his_members.head()

,1980-03-31,1980-06-30,1980-09-30,1980-12-31,1981-03-31,1981-06-30,1981-09-30,1981-12-31,1982-03-31,1982-06-30,...,2020-09-30,2020-12-31,2021-03-31,2021-06-30,2021-09-30,2021-12-31,2022-03-31,2022-06-30,2022-09-30,2022-10-20
0,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,0111145D UN Equity,...,1981718D UW Equity,2061517D UW Equity,2061517D UW Equity,2150994D UW Equity,2150994D UW Equity,1950546D US Equity,2150994D UW Equity,2150994D UW Equity,A UN Equity,A UN Equity
1,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,0118113D UN Equity,...,2061517D UW Equity,2078185D UN Equity,2150994D UW Equity,9990213D UN Equity,9990213D UN Equity,2150994D UW Equity,9990213D UN Equity,9990213D UN Equity,AA UN Equity,AA UN Equity
2,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,0203524D UN Equity,...,2078185D UN Equity,2150994D UW Equity,9990213D UN Equity,9990253D UN Equity,9990253D UN Equity,9990213D UN Equity,9990253D UN Equity,A UN Equity,AAL UW Equity,AAL UW Equity
3,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,0226226D UN Equity,...,2150994D UW Equity,9990213D UN Equity,9990253D UN Equity,9990527D UW Equity,9990527D UW Equity,9990253D UN Equity,9990527D UW Equity,AA UN Equity,AAP UN Equity,AAP UN Equity
4,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,0544749D UN Equity,...,9990213D UN Equity,9990253D UN Equity,9990620D UN Equity,A UN Equity,A UN Equity,9990527D UW Equity,A UN Equity,AAL UW Equity,ACGL UW Equity,ACGL UW Equity


In [3]:
ave_ecoc = pd.read_excel("../clean_data/ave_ecoc1_1110.xlsx", index_col=0)
FCFROIC_ADJ = pd.read_excel("../clean_data/FCFROIC_ADJ_1110.xlsx", index_col=0)
ave_ecoc.head()

,beta,name,MCALDT,TMBIDYTM,rf,coe,cost of equity,10y_avg
369,1.208248,A,1990-10-31,7.275046,0.072750,0.145245,0.145245,0.166387
370,1.182409,A,1990-11-30,7.180697,0.071807,0.142752,0.142752,0.166067
370,1.151294,A,1990-11-30,7.180697,0.071807,0.140885,0.140885,0.165700
371,1.154007,A,1990-12-31,6.580788,0.065808,0.135048,0.135048,0.165267
372,1.099094,A,1991-01-31,6.325590,0.063256,0.129202,0.129202,0.164808


In [11]:
ave_ecoc.index = ave_ecoc["MCALDT"]

In [6]:
FCFROIC_ADJ.head()

,Global Company Key,Data Date,Fiscal Year,Fiscal Quarter,Ticker Symbol,FCFROIC,ROA,ROE,FCFROIC_ADJ,filtered,avg_fcfroic,avg_fcfroic_ajd
0,1045,1963-06-30,1963,2.0,AAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1045,1963-09-30,1963,3.0,AAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1045,1963-12-31,1963,4.0,AAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1045,1964-03-31,1964,1.0,AAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1045,1964-06-30,1964,2.0,AAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
fcfroic_grouped = FCFROIC_ADJ.groupby("Data Date")

In [9]:
pd.to_datetime(q_ls)

DatetimeIndex(['1980-03-31', '1980-06-30', '1980-09-30', '1980-12-31',
               '1981-03-31', '1981-06-30', '1981-09-30', '1981-12-31',
               '1982-03-31', '1982-06-30',
               ...
               '2020-09-30', '2020-12-31', '2021-03-31', '2021-06-30',
               '2021-09-30', '2021-12-31', '2022-03-31', '2022-06-30',
               '2022-09-30', '2022-10-20'],
              dtype='datetime64[ns]', length=172, freq=None)

In [12]:
q_ls = his_members.columns
q_ls_d = pd.to_datetime(q_ls)

ls = []

for i in range(his_members.shape[1]-1):
    m = his_members.iloc[:, i].dropna()
    g = FCFROIC_ADJ[FCFROIC_ADJ["Data Date"] == m.name]

    m = m.str.extract(r'(\w+)')[0].values
    subset = g[g["Ticker Symbol"].isin(m)]
    subset = subset.dropna(subset=["FCFROIC_ADJ"])
    if subset.shape[0] > 0:
        ecoc_sub = ave_ecoc[(ave_ecoc.index >= q_ls_d[i]) & (
            ave_ecoc.index < q_ls_d[i+1])].groupby("name").mean()
        subset = subset.merge(
            ecoc_sub, left_on="Ticker Symbol", right_index=True)
        subset["ind"] = 0
        subset.loc[(subset["avg_fcfroic_ajd"] >
                    subset["10y_avg"]), "ind"] = 1
        ls.append(subset)

quality = pd.concat(ls, ignore_index=True)

In [13]:
quality

,Global Company Key,Data Date,Fiscal Year,Fiscal Quarter,Ticker Symbol,FCFROIC,ROA,ROE,FCFROIC_ADJ,filtered,avg_fcfroic,avg_fcfroic_ajd,beta,TMBIDYTM,rf,coe,cost of equity,10y_avg,ind
0,9299,1987-12-31,1987,4.0,R,-0.154917,-0.004939,-0.020139,-0.154917,-0.154917,NaN,NaN,1.312289,5.765592,0.057656,0.136393,0.136393,0.155711,0
1,10614,1987-12-31,1987,4.0,GL,0.225736,-0.024744,NaN,0.225736,0.225736,NaN,NaN,0.989231,5.765592,0.057656,0.117010,0.117010,0.138733,0
2,9299,1988-03-31,1988,1.0,R,-0.044759,-0.005749,-0.023584,-0.044759,-0.044759,NaN,NaN,1.226900,6.205839,0.062058,0.135672,0.135672,0.154766,0
3,10614,1988-03-31,1988,1.0,GL,0.063481,0.016048,NaN,0.063481,0.063481,NaN,NaN,1.041764,6.205839,0.062058,0.124564,0.124564,0.136645,0
4,9299,1988-06-30,1988,2.0,R,-0.071586,0.012932,0.052824,-0.071586,-0.071586,NaN,NaN,1.187811,6.998372,0.069984,0.141252,0.141252,0.154335,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30869,199356,2021-12-31,2021,4.0,TRIP,0.031386,-0.052163,-0.151611,0.031386,0.031386,0.109387,0.109387,1.376103,0.050698,0.000507,0.083073,0.083073,0.054006,1
30870,260774,2021-12-31,2021,4.0,CBRE,0.183715,0.049038,0.118106,0.183715,0.183715,0.012602,0.012602,1.396285,0.050698,0.000507,0.084284,0.084284,0.114787,0
30871,315318,2021-12-31,2021,4.0,ESI,0.062421,-0.024881,-0.051197,0.062421,0.062421,NaN,NaN,1.356493,0.050698,0.000507,0.081897,0.081897,0.121450,0
30872,316056,2021-12-31,2021,4.0,ALLE,0.202411,-0.039820,-0.150675,0.202411,0.202411,NaN,NaN,1.028634,0.050698,0.000507,0.062225,0.062225,0.050331,0
